In [29]:
import pandas as pd
import re
import nltk
from collections import Counter

In [136]:
# load data
songs_data = pd.read_csv("data/top50_2018_2022.csv")
bad_words_data = pd.read_csv("data/bad-words.csv")
bad_words_data

,jigaboo
0,mound of venus
1,asslover
2,s&m
3,queaf
4,whitetrash
...,...
1611,cocky
1612,transsexual
1613,unfuckable
1614,bestiality


In [95]:
# primary artists -> separate by comma
songs_data["Primary Artists"] = songs_data["Primary Artists"].str.replace(" & ",",")
songs_data["Primary Artists"] = songs_data["Primary Artists"].str.replace(", ", ",")
songs_data["Primary Artists"] = songs_data["Primary Artists"].str.split(",")

In [96]:
songs_data["Primary Artists"]

0                             [Drake]
1      [Cardi B, Bad Bunny, J Balvin]
2                             [Drake]
3                       [Post Malone]
4                             [Drake]
                    ...              
245                      [Juice WRLD]
246       [Baby Keem, Kendrick Lamar]
247                        [Rod Wave]
248             [Megan Thee Stallion]
249                    [Armani White]
Name: Primary Artists, Length: 250, dtype: object

In [22]:
def preprocess_lyrics(lyrics):
    # Remove punctuation
    lyrics = re.sub('[^a-zA-Z0-9\s]', '', lyrics)
    # Convert to lowercase
    lyrics = lyrics.lower()
    # Tokenize into individual words
    words = nltk.word_tokenize(lyrics)
    return words

In [126]:
def word_list_to_df(words):
    df = pd.DataFrame.from_dict(Counter(used_words), orient="index", columns=["count"]).reset_index()
    df.index.name = "word"
    return df

In [127]:
# a table of ALL words (independent of any other variable)
used_words = []
for lyrics in songs_data["Lyrics"]:
    for word in preprocess_lyrics(lyrics):
        used_words.append(word)

all_words_df = word_list_to_df(used_words)

In [129]:
# a table of words FOR EACH year
words_year_dfs = {}
for year in songs_data["year"].unique():
    words_year_dfs[year] = []

# combine all lyrics of a year
year_lyrics_df = songs_data.groupby("year")["Lyrics"].apply(' '.join).reset_index()
for year in words_year_dfs.keys():
    lyrics_of_that_year = year_lyrics_df.loc[year_lyrics_df["year"]==year, "Lyrics"].values[0]
    words=preprocess_lyrics(lyrics_of_that_year)
    words_year_dfs[year] = word_list_to_df(words)

In [133]:
words_year_dfs[2020]

,index,count
word,,
0,and,1978
1,they,912
2,wishin,30
3,on,1903
4,me,2294
...,...,...
11243,mitt,1
11244,kimbo,1
11245,gen3,1


In [134]:
# a table of words FOR EACH primary artist
words_artist_dfs = {}
# if multiple primary artists: EXPLODE!! (means per duplicate row for each artist)
exploded_songs_data = songs_data.explode('Primary Artists')
for artist in exploded_songs_data["Primary Artists"].unique():
    words_artist_dfs[artist] = []

# combine all lyrics of an artist
artist_lyrics_df = explored_songs_data.groupby("Primary Artists")["Lyrics"].apply(' '.join).reset_index()
for artist in words_artist_dfs.keys():
    lyrics_of_that_artist = artist_lyrics_df.loc[artist_lyrics_df["Primary Artists"]==artist, "Lyrics"].values[0]
    words=preprocess_lyrics(lyrics_of_that_artist)
    words_artist_dfs[artist] = word_list_to_df(words)

In [135]:
words_artist_dfs["Post Malone"]

,index,count
word,,
0,and,1978
1,they,912
2,wishin,30
3,on,1903
4,me,2294
...,...,...
11243,mitt,1
11244,kimbo,1
11245,gen3,1


In [139]:
for i,j in enumerate(words_year_dfs):
    print(i)

0
1
2
3
4


In [142]:
# create swear word column to each
swear_all_words_df = all_words_df
swear_all_words_df["swearword"] = all_words_df["index"].isin(bad_words_data["jigaboo"])

swear_words_year_dfs = words_year_dfs
for k in words_year_dfs.keys():
    swear_words_year_dfs[k]["swearword"] = words_year_dfs[k]["index"].isin(bad_words_data["jigaboo"])

swear_words_artist_dfs = words_artist_dfs
for k in words_artist_dfs.keys():
    swear_words_artist_dfs[k]["swearword"] = words_artist_dfs[k]["index"].isin(bad_words_data["jigaboo"])

In [141]:
swear_words_year_dfs[2020]

,index,count,swearword
word,,,
0,and,1978,False
1,they,912,False
2,wishin,30,False
3,on,1903,False
4,me,2294,False
...,...,...,...
11243,mitt,1,False
11244,kimbo,1,False
11245,gen3,1,False
